Word2Vec



In [11]:
from gensim.models import Word2Vec
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
import gensim.downloader as api
import gensim


import pandas as pd
import numpy as np



In [10]:
import sys
!{sys.executable} -m pip install wikipedia2vec

  Using cached wikipedia2vec-1.0.5.tar.gz (1.2 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for wikipedia2vec
Failed to build wikipedia2vec
  Running setup.py install for wikipedia2vec: started
  Running setup.py install for wikipedia2vec: finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [115 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-310
      creating build\lib.win-amd64-cpython-310\wikipedia2vec
      copying wikipedia2vec\cli.py -> build\lib.win-amd64-cpython-310\wikipedia2vec
      copying wikipedia2vec\__init__.py -> build\lib.win-amd64-cpython-310\wikipedia2vec
      creating build\lib.win-amd64-cpython-310\wikipedia2vec\utils
      copying wikipedia2vec\utils\__init__.py -> build\lib.win-amd64-cpython-310\wikipedia2vec\utils
      creating build\lib.win-amd64-cpython-310\wikipedia2vec\utils\sentence_detector
      copying wikipedia2vec\utils\sentence_detector\__init__.py -> build\lib.win-amd64-cpython-310\wikipedia2vec\utils\sentence_detector
      creating build\lib.win-amd64-cpython-310\wikipedia2vec\utils\tokenizer
      copying wiki

In [12]:
model = gensim.models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True)

In [13]:
from sklearn.metrics.pairwise import cosine_similarity

def get_cosine_similarity(feature_vec_1, feature_vec_2):
    return cosine_similarity(feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1))[0][0]

Read the files and create lists of the classes

In [14]:
img_net = pd.read_csv("../files/ImageNet.csv")
img_net_list = img_net["Class Name"].tolist()

dbpedia = pd.read_csv("../files/dbpedia_classes.csv")
db_list = dbpedia["label"].tolist()

In [15]:
dataframe = pd.DataFrame(index =db_list ,columns =img_net_list)

1. For ImageNet, we take the mean over all the elements in the classList, for mulitwords we take the average over all the words

In [16]:
zero_vec = np.zeros(300, dtype=int)
def get_mean_vector_new(word2vec_model, words):
    # remove out-of-vocabulary words
    #words = [word for word in words if word in word2vec_model.key_to_index]
    process_words = []
    for el in words:
        if el in model.key_to_index:
            process_words.append(model[el])
        else:
            multi = el.split(",")
            #new_val = []
            arr = np.ndarray(shape=(len(multi),300))
            i = 0
            for elem in multi:
                elem = elem.strip()
                mult_words = elem.split(" ")
                if len(mult_words) > 1:
                    mult_vec = np.ndarray(shape=(len(mult_words),300))
                    j = 0
                    for e in mult_words:
                        if e in model.key_to_index:
                            mult_vec[j] = model[e]
                            j += 1
                        else:
                            mult_vec[j] = zero_vec
                            j += 1
                    arr[i] = np.mean(mult_vec, axis = 0)
                    i += 1
                elif elem in model.key_to_index:
                    #new_val.append(model[elem])
                    arr[i] = model[elem]
                    i += 1
                else:
                    #new_val.append(zero_vec)
                    arr[i] = zero_vec
                    i += 1
            process_words.append(np.mean(arr, axis=0))#geht das so mit dem np.mean PROBLEM we receive one value but should receive a matrix                

    return process_words

In [18]:
el_img_vec = get_mean_vector_new(model, img_net_list) # is size 1000 
#el_img_vec[48][57] = 0.0
zero_vec = np.zeros(300, dtype=int)

for i in range(0,len(el_img_vec)):
    for el_db in db_list:
        a = db_list.index(el_db) 
        multi_db = el_db.split(" ")
        if len(multi_db) > 1:
            m_db = np.ndarray(shape=(len(multi_db),300))
            x = 0
            for e in multi_db:
                if e in model.key_to_index:
                    m_db[x] = model[e]
                    x += 1
                else:
                    m_db[x] = zero_vec
                    x += 1
            dbpedia = np.mean(m_db, axis=0)
            
            sim = get_cosine_similarity(el_img_vec[i],dbpedia)
            dataframe.iat[a,i] = sim            
        else:
            try:
                sim = get_cosine_similarity(el_img_vec[i],model[el_db])
                dataframe.iat[a,i] = sim                
            except:
                dataframe.iat[a,i] = "Not Present"  # this happens if the dbpedia word is not present in the model


In [19]:
dataframe

,"tench, Tinca tinca","goldfish, Carassius auratus","great white shark, white shark, man-eater, man-eating shark, Carcharodon caharias',","tiger shark, Galeocerdo cuvieri","hammerhead, hammerhead shark","electric ray, crampfish, numbfish, torpedo",stingray,cock,hen,"ostrich, Struthio camelus",...,"buckeye, horse chestnut, conker",coral fungus,agaric,gyromitra,"stinkhorn, carrion fungus",earthstar,"hen-of-the-woods, hen of the woods, Polyporus frondosus, Grifola frondosa",bolete,"ear, spike, capitulum","toilet tissue, toilet paper, bathroom tissue"
company,-0.03251,0.015643,0.036344,0.035145,0.008221,0.059409,0.046758,-0.020425,-0.005486,0.039336,...,0.000198,0.068306,0.0,0.0,0.046826,0.0,0.026019,0.044094,0.035107,0.115263
activity,0.098292,0.035124,0.021873,0.02476,0.070632,0.044903,-0.058751,0.075186,0.053053,-0.0168,...,0.030169,0.062173,0.0,0.0,0.111502,0.0,0.130787,0.016908,0.125571,0.083766
name,0.038352,0.164622,0.085025,0.070638,0.027696,0.007783,-0.016919,0.03791,0.017966,0.012867,...,0.181275,0.03067,0.0,0.0,0.165809,0.0,0.052724,0.142739,0.102462,0.084176
person,0.030184,0.197448,0.194279,0.175427,0.150168,0.124157,0.134935,0.069695,0.106013,0.083804,...,0.133522,0.131261,0.0,0.0,0.181539,0.0,0.141329,0.140267,0.146012,0.174744
actor,0.131938,0.126792,0.136068,0.150914,0.113341,0.037481,0.135175,0.075059,0.046779,0.125658,...,0.087242,0.03105,0.0,0.0,0.05405,0.0,0.063014,0.161637,0.006914,0.111038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wind motor,0.027858,0.084458,0.108734,0.092826,0.187492,0.281067,0.071656,0.08422,0.089732,0.188199,...,0.165033,0.096989,0.0,0.0,0.079514,0.0,0.141445,0.038754,0.144252,0.080255
Windmill,0.24404,0.132538,0.118124,0.074022,0.159601,0.024737,0.033151,0.123957,0.147012,0.078618,...,0.15909,0.096168,0.0,0.0,0.048459,0.0,0.181245,0.113307,0.068772,0.094144
wine,0.049161,0.153337,0.147769,0.114043,0.075134,0.047055,0.109544,0.07916,0.164966,0.139752,...,0.249659,0.193899,0.0,0.0,0.190574,0.0,0.173993,0.313743,0.039106,0.131012
woman,-0.013632,0.177545,0.228693,0.210612,0.161871,0.096435,0.147436,0.102571,0.149326,0.134999,...,0.219156,0.138435,0.0,0.0,0.167615,0.0,0.216297,0.208203,0.103132,0.246693


In [ ]:
a = dataframe
a = dataframe.replace("Not Present", np.nan)
a =a.replace(0.0, np.nan)

#Now we remove all columns where we have only NaN values
nan = a.dropna(axis=1, how="all")
highest_values_nan = nan.apply(lambda s: s.abs().nlargest(5).index.tolist(), axis=0)
highest_values_nan

,"tench, Tinca tinca","goldfish, Carassius auratus","great white shark, white shark, man-eater, man-eating shark, Carcharodon caharias',","tiger shark, Galeocerdo cuvieri","hammerhead, hammerhead shark","electric ray, crampfish, numbfish, torpedo",stingray,cock,hen,"ostrich, Struthio camelus",...,corn,acorn,"hip, rose hip, rosehip","buckeye, horse chestnut, conker",coral fungus,"stinkhorn, carrion fungus","hen-of-the-woods, hen of the woods, Polyporus frondosus, Grifola frondosa",bolete,"ear, spike, capitulum","toilet tissue, toilet paper, bathroom tissue"
0,fish,fish,reptile,reptile,mammal,rocket engine,reptile,bird,bird,animal,...,farmer,fern,ligament,fern,fungus,fungus,bird,grape,storm surge,body of water
1,lake,reptile,mammal,mammal,fish,rocket,mammal,hollywood cartoon,cat,reptile,...,grape,conifer,bone,horse,bacteria,insect,cat,cycad,brain,bacteria
2,species,cat,fish,fish,reptile,weapon,fish,cat,animal,horse,...,coal pit,moss,muscle,conifer,insect,bacteria,garden,fern,artery,enzyme
3,snooker world champion,crustacean,crustacean,cat,crustacean,asteroid,crustacean,naruto character,dog,bird,...,flowering plant,insect,nerve,horse race,moss,moss,animal,moss,bone,blood vessel
4,Windmill,dog,surfer,animal,amphibian,automobile engine,surfer,darts player,horse,cat,...,fungus,flowering plant,club moss,horse rider,species,cycad,dog,crustacean,device,anatomical structure


In [ ]:
from pathlib import Path  

filepath = Path('../files/word_embeddings/word_2_vec_top5_mean.csv')  

filepath.parent.mkdir(parents=True, exist_ok=True)  

highest_values_nan.to_csv(filepath, index=False)  

Method 2: Now we take the max value if we have an array

In [37]:
#gives in a list of arrays and returns the mean
def preprocess_array(model,mult_list, split_criterion:str):
    output_vec = []
    zero_vec = np.zeros(300, dtype=float)
    for el in mult_list.split(split_criterion):
        if el in model.key_to_index:
            output_vec.append(model[el])
        else:
            output_vec.append(zero_vec)
    return np.mean(output_vec, axis = 0)

In [40]:
dataframe_max = pd.DataFrame(index =db_list ,columns =img_net_list)

In [44]:
#el_img_vec = get_mean_vector_new(model, class_list) # is size 1000 

zero_vec = np.zeros(300, dtype=int)

for el_img in img_net_list:
    for el_db in db_list:
        a = db_list.index(el_db)
        i = img_net_list.index(el_img) 
        multi_db = el_db.split(" ")
        if len(multi_db) > 1:
            m_db = np.ndarray(shape=(len(multi_db),300))
            x = 0
            for e in multi_db:
                if e in model.key_to_index:
                    m_db[x] = model[e]
                    x += 1
                else:
                    m_db[x] = zero_vec
                    x += 1
            dbpedia = np.mean(m_db, axis=0)
            sim = -1
            for el in el_img.split(","):
                el = preprocess_array(model, el, " ")
            
                val = get_cosine_similarity(el,dbpedia)
                if val > sim:
                    sim = val
            dataframe_max.iat[a,i] = sim            
        else:
            try:
                sim = -1
                for elem in el_img.split(","):

                    elem = preprocess_array(model, elem, " ")
            
                    val = get_cosine_similarity(elem,model[el_db])
                    if val > sim:
                        sim = val
                #sim = get_cosine_similarity(el_img_vec[i],model[el_db])
                dataframe_max.iat[a,i] = sim                
            except:
                dataframe_max.iat[a,i] = "Not Present" 

In [45]:
dataframe_max

,"tench, Tinca tinca","goldfish, Carassius auratus","great white shark, white shark, man-eater, man-eating shark, Carcharodon caharias',","tiger shark, Galeocerdo cuvieri","hammerhead, hammerhead shark","electric ray, crampfish, numbfish, torpedo",stingray,cock,hen,"ostrich, Struthio camelus",...,"buckeye, horse chestnut, conker",coral fungus,agaric,gyromitra,"stinkhorn, carrion fungus",earthstar,"hen-of-the-woods, hen of the woods, Polyporus frondosus, Grifola frondosa",bolete,"ear, spike, capitulum","toilet tissue, toilet paper, bathroom tissue"
company,0.0,0.015643,0.040047,0.035145,0.017426,0.053889,0.046758,-0.020425,-0.005486,0.039336,...,0.071817,0.068306,0.0,0.0,0.085171,0.0,0.026019,0.044094,0.081949,0.132506
activity,0.098292,0.035124,0.033281,0.02476,0.074432,0.070356,-0.058751,0.075186,0.053053,0.0,...,0.07535,0.062173,0.0,0.0,0.11922,0.0,0.130787,0.016908,0.194163,0.11094
name,0.038352,0.164622,0.111474,0.070638,0.036323,0.011278,-0.016919,0.03791,0.017966,0.012868,...,0.183086,0.03067,0.0,0.0,0.237197,0.0,0.052724,0.142739,0.115416,0.139728
person,0.030184,0.197448,0.194613,0.175427,0.172406,0.10857,0.134935,0.069695,0.106013,0.083804,...,0.184806,0.131261,0.0,0.0,0.202575,0.0,0.141329,0.140267,0.1479,0.169184
actor,0.131938,0.126792,0.142096,0.150914,0.133654,0.032256,0.135175,0.075059,0.046779,0.125658,...,0.138943,0.03105,0.0,0.0,0.098885,0.0,0.063014,0.161637,0.043155,0.11201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wind motor,0.027858,0.084458,0.124024,0.092826,0.196164,0.345437,0.071656,0.08422,0.089732,0.188199,...,0.185186,0.096989,0.0,0.0,0.08996,0.0,0.141445,0.038754,0.172234,0.087656
Windmill,0.24404,0.132538,0.138522,0.074022,0.161669,0.084572,0.033151,0.123957,0.147012,0.078618,...,0.221645,0.096168,0.0,0.0,0.065382,0.0,0.181245,0.113307,0.093221,0.133364
wine,0.049161,0.153337,0.167979,0.114043,0.095882,0.041781,0.109544,0.07916,0.164966,0.139752,...,0.239805,0.193899,0.0,0.0,0.193801,0.0,0.173993,0.313743,0.050198,0.12813
woman,0.0,0.177545,0.25554,0.210613,0.181486,0.112927,0.147436,0.102571,0.149326,0.134999,...,0.275723,0.138435,0.0,0.0,0.206066,0.0,0.216297,0.208203,0.087762,0.235581


In [50]:
a = dataframe_max
a = dataframe_max.replace("Not Present", np.nan)
a =a.replace(0.0, np.nan)

#Now we remove all columns where we have only NaN values
nan = a.dropna(axis=1, how="all")
highest_values_nan_max = nan.apply(lambda s: s.abs().nlargest(5).index.tolist(), axis=0)
highest_values_nan_max

,"tench, Tinca tinca","goldfish, Carassius auratus","great white shark, white shark, man-eater, man-eating shark, Carcharodon caharias',","tiger shark, Galeocerdo cuvieri","hammerhead, hammerhead shark","electric ray, crampfish, numbfish, torpedo",stingray,cock,hen,"ostrich, Struthio camelus",...,corn,acorn,"hip, rose hip, rosehip","buckeye, horse chestnut, conker",coral fungus,"stinkhorn, carrion fungus","hen-of-the-woods, hen of the woods, Polyporus frondosus, Grifola frondosa",bolete,"ear, spike, capitulum","toilet tissue, toilet paper, bathroom tissue"
0,fish,fish,reptile,reptile,reptile,solar eclipse,reptile,bird,bird,animal,...,farmer,fern,ligament,horse,fungus,fungus,bird,grape,storm surge,bacteria
1,lake,reptile,mammal,mammal,mammal,rocket,mammal,hollywood cartoon,cat,reptile,...,grape,conifer,bone,horse rider,bacteria,insect,cat,cycad,brain,enzyme
2,species,cat,fish,fish,fish,rocket engine,fish,cat,animal,horse,...,coal pit,moss,grape,horse race,insect,bacteria,garden,fern,bone,body of water
3,snooker world champion,crustacean,crustacean,cat,crustacean,Wind motor,crustacean,naruto character,dog,bird,...,flowering plant,insect,polysaccharide,horse trainer,moss,moss,animal,moss,sound,brain
4,Windmill,dog,surfer,animal,amphibian,electrical substation,surfer,darts player,horse,cat,...,fungus,flowering plant,ginkgo,jockey (horse racer),species,cycad,dog,crustacean,camera,anatomical structure


In [51]:
from pathlib import Path  

filepath = Path('../files/word_embeddings/word_2_vec_top5_max.csv')  

filepath.parent.mkdir(parents=True, exist_ok=True)  

highest_values_nan_max.to_csv(filepath, index=False)  